## Test run for pglmm from phyr R package

The plgmm functions allows to use phylo class objects from ape to account for species phylogeny in generalized mixte models to predict binary variable from binary, counts or continuous predictors

Ressources:
- https://besjournals.onlinelibrary.wiley.com/doi/full/10.1111/2041-210X.13471
- https://daijiang.github.io/phyr/articles/phyr_example_empirical.html
- https://daijiang.github.io/phyr/reference/pglmm.html

In this Jupyter Notebook I am trying to see if this could be a good solution to incorporate phylogeny information when modeling the binary FSA with a discrete predictor (turned into a binary predictor).
Here are two exemples to evaluate if this could be a solution

In [1]:
install.packages('phyr', repos='http://cran.us.r-project.org')

also installing the dependencies ‘deldir’, ‘png’, ‘jpeg’, ‘interp’, ‘mvtnorm’, ‘latticeExtra’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [8]:
library(ape)
library(ggtree)
library(phyr)
library(tidyverse)

── Attaching packages ──────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.5.0 
✔ readr   2.1.3      ✔ forcats 0.5.2 
Warning message:
“package ‘stringr’ was built under R version 4.2.3”
── Conflicts ─────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks ggtree::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
## Data import
data_disc_traits=read.csv('data_tree_trait_210species_discrete.csv')
tree=read.tree('tree_for_trait_210.nwk') #phylogenetic tree that contains phylogenetic relationships for al 210 species


### Predicting FSA status from presence/absence of gene

Here I am using the gene endoglucanase whcih is the genes that came up with the most significant p-value when running a Fisher test to test whether presence of FSA and presence of this gene are independent

In [4]:
# Data formating for pglmm

data_genes=data_disc_traits[,c('tip.label','FSA','endoglucanase9_count')]%>%na.omit()

data_genes_simple=data_genes
data_genes_simple$endoglucanase9_count[data_genes_simple$endoglucanase9_count>=1]=1 # setting up gene as binary information

data_genes_simple$FSA[data_genes_simple$FSA=='Yes']=1
data_genes_simple$FSA[data_genes_simple$FSA=='No']=0

data_genes_simple$FSA=as.numeric(data_genes_simple$FSA)


# phylo class object preparation that is used as the input matrix to take into account the random effecst assocaited with the phylogeny
tree_tip=intersect(tree$tip.label,data_genes_simple$tip.label)
tree_pruned=keep.tip(tree, tree_tip)


In [5]:
# pglmm model that take into account random effect from phylogeny


mod_test=pglmm(FSA ~ endoglucanase9_count + (1|tip.label__) + (endoglucanase9_count|tip.label__),
               data=data_genes_simple, 
               cov_ranef=list(tip.label=tree_pruned),
               family='binomial')

mod_test

# tip.label__ is what we used to specify an overall phylogenetic effect
# endoglucanase|tip.label__ is what we use to estimate the degree to which presence of FSA in species that have or don't have the gene endoglucanase has a phylogenetic signal.

as(<matrix>, "dgTMatrix") is deprecated since Matrix 1.5-0; do as(as(as(., "dMatrix"), "generalMatrix"), "TsparseMatrix") instead



Generalized linear mixed model for binomial data fit by restricted maximum likelihood

Call:FSA ~ endoglucanase9_count


Random effects:
                                  Variance  Std.Dev
1|tip.label                      1.066e-06 0.001033
1|tip.label__                    2.289e-01 0.478426
endoglucanase9_count|tip.label   8.897e-06 0.002983
endoglucanase9_count|tip.label__ 8.363e-06 0.002892

Fixed effects:
                        Value Std.Error  Zscore  Pvalue    
(Intercept)          -1.08555   0.77556 -1.3997 0.16160    
endoglucanase9_count  1.88249   0.57011  3.3020 0.00096 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


If I understand this well. This is also telling us that the presence of the gene enndoglucanase is a predictor of FSA being present.  The random effects indicates that the strongest effect is an overall effect of the phylogeny.

### Predicting FSA status with discrete data

Here I am using the simpler discrete trait we have in our dataset: Ascoma_development. This trait is characterized by 3 levels: Absent, ... and ...
The fisher exact test returns that Ascoma development and FSA status are not independent and post-hoc test indicates that absence of ascoma correlated with absence of FSA - In turns out that all the species without ascoma are yeast species, potentially highlighting that this is mostly because of phylogeny and maybe not a real association between the trait and FSA

In [9]:
# Data formating for pglmm


data_disc_ex=data_disc_traits[,c('tip.label','FSA','ascoma_development')]%>%na.omit()

data_ex=data_disc_ex
data_ex$FSA[data_ex$FSA=='Yes']=1
data_ex$FSA[data_ex$FSA=='No']=0
data_ex$FSA=as.numeric(data_ex$FSA)

#Now I turn each level into a binary predictor
data_ex$bin=1
data_ex_bin=spread(data_ex,ascoma_development,bin)  %>% replace(is.na(.), 0)

data_ex_bin




tip.label,FSA,Absent,Ascohymenial,Ascolocular
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Alyxoria_varia,0,0,1,0
Aspergillus_nidulans,1,0,1,0
Bipolaris_maydis,0,0,0,1
Bipolaris_sorokiniana,0,0,0,1
Blumeria_graminis,0,0,1,0
Botryosphaeria_dothidea,0,0,0,1
Ceratocystis_fimbriata,0,0,1,0
Claviceps_purpurea,0,0,1,0
Clavispora_lusitaniae,0,1,0,0


In [13]:
# phylo class object preparation that is used as the input matrix to take into account the random effecst assocaited with the phylogeny
tree_tip=intersect(tree$tip.label,data_ex_bin$tip.label)
tree_pruned=keep.tip(tree, tree_tip)

In [14]:
# pglmm model that take into account random effect from phylogeny


mod_test=pglmm(FSA ~ Absent + Ascohymenial + Ascolocular+
               (1|tip.label__) + (Absent|tip.label__) + (Ascohymenial|tip.label__) + 
               (Ascolocular|tip.label__),
               data=data_ex_bin, 
               cov_ranef=list(tip.label=tree_pruned),
               family='binomial')

mod_test

ERROR: Error in pglmm_internal_cpp(X = X, Y = Y, Zt = Zt, St = St, nested = nested, : Evaluation error: objective in x0 returns NA.


**Running the model like this returns an error highlighting that at some point the inv() function returns a singular matrix. Thhis is a consistent error I met whenever I am trying to include multiple levels in the model (3 or more).
We can maybe try running the model for each level individually**

In [15]:
# Running the pglmm model for each levels individually

#Absent
mod_test_absent=pglmm(FSA ~ Absent +
               (1|tip.label__) + (Absent|tip.label__),
               data=data_ex_bin, 
               cov_ranef=list(tip.label=tree_pruned),
               family='binomial')

mod_test_absent

Generalized linear mixed model for binomial data fit by restricted maximum likelihood

Call:FSA ~ Absent


Random effects:
                    Variance  Std.Dev
1|tip.label        1.902e-06 0.001379
1|tip.label__      3.395e-01 0.582651
Absent|tip.label   1.229e-04 0.011088
Absent|tip.label__ 1.510e+00 1.228739

Fixed effects:
               Value Std.Error  Zscore Pvalue
(Intercept) -0.11894   0.73685 -0.1614 0.8718
Absent      -1.86472   1.86281 -1.0010 0.3168


**Interestingly while fisher test not including phylogeny information highlighted that absence of ascoma is a predictor of FSA status, when taking phylogeny into account, this is no longer true. 
Also, we can see that the Absent|tip.label__ random effect is the greetest followed by 1|tip.label__, this, if I understand correctly, indicates that the strongest effects are phylogenetic?**

In [19]:
# Running the pglmm model for each levels individually

#Ascohymenial
mod_test_ascohym=pglmm(FSA ~ Ascohymenial +
               (1|tip.label__) + (Ascohymenial|tip.label__),
               data=data_ex_bin, 
               cov_ranef=list(tip.label=tree_pruned),
               family='binomial')

mod_test_ascohym



Generalized linear mixed model for binomial data fit by restricted maximum likelihood

Call:FSA ~ Ascohymenial


Random effects:
                          Variance  Std.Dev
1|tip.label              3.681e-02 0.191866
1|tip.label__            1.896e-01 0.435401
Ascohymenial|tip.label   3.718e-04 0.019283
Ascohymenial|tip.label__ 4.892e-05 0.006994

Fixed effects:
                Value Std.Error  Zscore  Pvalue  
(Intercept)  -1.88136   0.83702 -2.2477 0.02460 *
Ascohymenial  2.15568   1.00720  2.1403 0.03233 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [21]:

#Ascolocularl
mod_test_ascoloc=pglmm(FSA ~ Ascolocular +
               (1|tip.label__) + (Ascolocular|tip.label__),
               data=data_ex_bin, 
               cov_ranef=list(tip.label=tree_pruned),
               family='binomial')

mod_test_ascoloc

Generalized linear mixed model for binomial data fit by restricted maximum likelihood

Call:FSA ~ Ascolocular


Random effects:
                         Variance   Std.Dev
1|tip.label             4.209e-07 0.0006488
1|tip.label__           7.047e-01 0.8394691
Ascolocular|tip.label   1.158e+00 1.0762493
Ascolocular|tip.label__ 1.812e-05 0.0042573

Fixed effects:
               Value Std.Error  Zscore Pvalue
(Intercept) -0.63932   0.76331 -0.8376 0.4023
Ascolocular -1.58721   1.61049 -0.9855 0.3244


**Here, Ascohymenial ascoma development form appear to be significant predictor of FSA status when taking phylogeny into account, which was not that clear with Fisher test** 

Questions for Austin:
- because of singular matrices emerging when multiple levels are present, is treating each level as an independent predictor the best solution?